<a href="https://colab.research.google.com/github/kaiwei1016/ColabProjects/blob/main/Python/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2_%E6%AF%8F%E6%97%A5%E8%82%A1%E5%83%B9%E6%94%B6%E7%9B%A4%E8%B3%87%E8%A8%8A%2B%E7%B0%A1%E6%98%93%E9%81%B8%E8%82%A1%E5%B7%A5%E5%85%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1.爬取股價收盤資訊（輸入日期）








In [8]:
import requests
import pandas as pd
from io import StringIO

def crawl_price(date):  # 定義function
  datestr = date.strftime('%Y%m%d')  # 將date變成字串，再從網站上把指定日期的股價抓下來
  response = requests.post('https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date='+datestr+'&type=ALLBUT0999&response=csv&_=1681804028704')
  lines = response.text.split('\n')
  newlines = []  # 篩選出有17個欄位的資料加到newlines
  for line in lines:
    if len(line.split('",')) == 17: 
      newlines.append(line)  
  df = pd.read_csv(StringIO("\n".join(newlines).replace('=',''))) # 用\n把每一行連在一起，刪除多餘的等號"="，用StringIO轉成檔案，再用pd.read_csv來讀取
  df = df.astype(str)  # 將df中的元素都變成字串
  df = df.applymap(lambda s: s.replace(',', ''))  # 將字串中的逗號","都刪除（用到lambda input:output）
  df = df.set_index('證券代號')  # 把證券代號變成index欄位
  return df

import datetime
df = crawl_price(datetime.datetime(2023,4,18))  # 輸入要查詢的日期
df

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
證券代號,,,,,,,,,,,,,,,,
0050,元大台灣50,5183750,8973,621931940,120.35,120.40,119.75,119.95,-,0.4,119.90,87,119.95,38,0.00,nan
0051,元大中型100,60282,172,3536707,58.85,58.85,58.50,58.50,-,0.3,58.45,1,58.55,1,0.00,nan
0052,富邦科技,303821,694,32574316,107.50,107.55,106.80,107.15,-,0.4,107.15,1,107.30,3,0.00,nan
0053,元大電子,31350,692,1881509,60.05,60.15,60.00,60.15,+,0.05,60.05,1,60.50,15,0.00,nan
0055,元大MSCI金融,130524,264,2929046,22.42,22.49,22.35,22.42,,0.0,22.40,2,22.43,6,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,新麗,88647,123,1811092,20.45,20.45,20.40,20.40,-,0.05,20.40,11,20.45,8,9.62,nan
9945,潤泰新,6141106,4504,218838483,35.90,35.95,35.40,35.50,-,0.45,35.45,30,35.50,38,12.24,nan
9946,三發地產,1229703,524,16705872,13.55,13.90,13.30,13.60,+,0.15,13.60,35,13.65,16,31.63,nan


# 2.簡易選股工具（計算紅K棒長度來選股）



In [3]:
df = df.apply(lambda s: pd.to_numeric(s, errors='coerce'))  # 將df中的元素從字串變成數字
df.dropna(axis=1, how='all', inplace=True)  # 刪除不是數字的資料
close_open = df['收盤價'] / df['開盤價']  # 紅棒長度小於1代表股價跌，大於1代表股票漲
df[close_open > 1.05]  # 選出收盤比開盤高5%以上的股票

,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
證券代號,,,,,,,,,,,,,
1533,6885026,3263,401376763,55.00,58.60,54.90,58.60,5.30,58.60,102,NaN,0,146.50
1538,928651,55,9240030,9.98,10.80,9.96,10.75,0.89,10.30,1,10.75,9,0.00
1720,10398273,7085,706936525,65.10,69.90,64.90,68.60,4.60,68.50,20,68.60,23,15.04
2314,14842192,7424,738183558,46.50,51.10,45.55,51.10,4.60,51.10,10737,NaN,0,0.00
2495,12926325,5478,334167670,25.00,26.70,24.70,26.70,2.40,26.70,8790,NaN,0,34.23
3004,8918736,5873,994774184,107.50,115.50,103.00,115.50,10.50,115.50,1000,NaN,0,40.53
3057,1357769,774,16925774,11.80,12.85,11.70,12.85,1.15,12.85,1242,NaN,0,428.33
3704,18318882,9738,783813254,40.80,43.90,40.10,43.90,3.95,43.90,25868,NaN,0,10.66
6606,2537786,1434,78984614,29.00,31.80,28.95,31.80,2.85,31.80,51,NaN,0,21.49



```



# 3.儲存到SQLite資料庫

In [10]:
import sqlite3
conn = sqlite3.connect('mydata')  #連結SQLite
df.to_sql('daily_price', conn, if_exists='replace') # 存檔/覆蓋
df = pd.read_sql('select * from daily_price', conn, index_col=['證券代號']) # 讀檔
df.head()

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
證券代號,,,,,,,,,,,,,,,,
0050,元大台灣50,5183750,8973,621931940,120.35,120.40,119.75,119.95,-,0.4,119.90,87,119.95,38,0.00,nan
0051,元大中型100,60282,172,3536707,58.85,58.85,58.50,58.50,-,0.3,58.45,1,58.55,1,0.00,nan
0052,富邦科技,303821,694,32574316,107.50,107.55,106.80,107.15,-,0.4,107.15,1,107.30,3,0.00,nan
0053,元大電子,31350,692,1881509,60.05,60.15,60.00,60.15,+,0.05,60.05,1,60.50,15,0.00,nan
0055,元大MSCI金融,130524,264,2929046,22.42,22.49,22.35,22.42,,0.0,22.40,2,22.43,6,0.00,nan


# 4.儲存到csv檔

In [13]:
df.to_csv('daily_price.csv', encoding='utf_8_sig')  # 儲存成csv檔案（方便用excel打開）
df = pd.read_csv('daily_price.csv', index_col=['證券代號'])  #讀檔
df.head()

,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Unnamed: 16
證券代號,,,,,,,,,,,,,,,,
0050,元大台灣50,5183750,8973,621931940,120.35,120.40,119.75,119.95,-,0.40,119.90,87,119.95,38,0.0,NaN
0051,元大中型100,60282,172,3536707,58.85,58.85,58.50,58.50,-,0.30,58.45,1,58.55,1,0.0,NaN
0052,富邦科技,303821,694,32574316,107.50,107.55,106.80,107.15,-,0.40,107.15,1,107.30,3,0.0,NaN
0053,元大電子,31350,692,1881509,60.05,60.15,60.00,60.15,+,0.05,60.05,1,60.50,15,0.0,NaN
0055,元大MSCI金融,130524,264,2929046,22.42,22.49,22.35,22.42,,0.00,22.40,2,22.43,6,0.0,NaN
